In [2]:
#import libraries & begin spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('ReadData').getOrCreate()
sc = spark.sparkContext
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
sqlContext = SQLContext(sc)
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col, translate
from pyspark.sql.types import IntegerType, FloatType, StringType, TimestampType, DoubleType
from pyspark.sql import functions as F
from pyspark.sql.functions import when, col, mean, desc, round
from pyspark.sql.types import StructType, StructField

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#create schema for the projects.csv
projects_schema = StructType([
    StructField("Project ID", StringType(), True),
    StructField("School ID", StringType(), True),
    StructField("Teacher ID", StringType(), True),
    StructField("Teacher Project Posted Sequence", IntegerType(), True),
    StructField("Project Type", StringType(), True),
    StructField("Project Title", StringType(), True),
    StructField("Project Essay", StringType(), True),
    StructField("Project Short Description", StringType(), True),
    StructField("Project Need Statement", StringType(), True),
    StructField("Project Subject Category Tree", StringType(), True),
    StructField("Project Subject Subcategory Tree", StringType(), True),
    StructField("Project Grade Level Category", StringType(), True),
    StructField("Project Resource Category", StringType(), True),
    StructField("Project Cost", DoubleType(), True),
    StructField("Project Posted Date", TimestampType(), True),
    StructField("Project Expiration Date", TimestampType(), True),
    StructField("Project Current Status", StringType(), True),
    StructField("Project Fully Funded Date", TimestampType(), True),
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#read in all .csv files from s3 bucket
donations_df = spark.read.csv("s3://donorchoose//Donations.csv", header =True, inferSchema = True)
donors_df = spark.read.csv("s3://donorchoose/Donors.csv", header =True, inferSchema = True)
teachers_df = spark.read.csv("s3://donorchoose/Teachers.csv", header =True, inferSchema = True)
projects_df = spark.read.csv("s3://donorchoose/Projects.csv", header =True, multiLine = True, schema = projects_schema, escape='"')
resources_df = spark.read.csv("s3://donorchoose/Resources.csv", header =True, inferSchema = True)
schools_df = spark.read.csv("s3://donorchoose/Schools.csv", header =True, inferSchema = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#join all dataframes for future data exploration and feature engineering
donor_donations = donations_df.join(donors_df, on = ['Donor ID'], how = 'inner')
projects_schools = projects_df.join(schools_df, on = ['School ID'], how = 'inner')
donor_projects = donor_donations.join(projects_schools, on = ['Project ID'], how = 'left')
projects_teachers = donor_projects.join(teachers_df, on = ['Teacher ID'], how = 'inner')
df = projects_teachers.join(resources_df, on = ['Project ID'], how = 'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#missing school cities were all Washington DC
df = df.fillna({'School City':'DC'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#rename all df column names to remove spaces
df_renamed = df.withColumnRenamed('Project ID', 'project_id').withColumnRenamed('Teacher ID', 'teacher_id')\
.withColumnRenamed('Donor ID', 'donor_id').withColumnRenamed('Donation ID', 'donation_id')\
.withColumnRenamed('Donation Included Optional Donation', 'optional_donation').withColumnRenamed('Donation Amount', 'donation_amount')\
.withColumnRenamed('Donor Cart Sequence', 'donor_cart_sequence').withColumnRenamed('Donation Received Date','donation_received_date')\
.withColumnRenamed('Donor City', 'donor_city').withColumnRenamed('Donor State', 'donor_state').withColumnRenamed('Donor Is Teacher', 'donor_is_teacher')\
.withColumnRenamed('Donor Zip', 'donor_zip').withColumnRenamed('School ID', 'school_id')\
.withColumnRenamed('Teacher Project Posted Sequence', 'teacher_project_posted_seq')\
.withColumnRenamed('Project ID', 'project_id').withColumnRenamed('Project Type', 'project_type')\
.withColumnRenamed('Project Title', 'project_title').withColumnRenamed('Project Subject Category Tree', 'project_cat')\
.withColumnRenamed('Project Subject Subcategory Tree', 'project_cat2').withColumnRenamed('Project Grade Level Category', 'project_grade_level_cat')\
.withColumnRenamed('Project Resource Category', 'project_resource_cat').withColumnRenamed('Project Cost', 'project_cost')\
.withColumnRenamed('Project Posted Date', 'project_posted_date').withColumnRenamed('Project Expiration Date', 'project_exp_date')\
.withColumnRenamed('Project Current Status', 'project_curr_stat').withColumnRenamed('Project Fully Funded Date', 'project_fully_funded_date')\
.withColumnRenamed('School Name', 'school_name').withColumnRenamed('School Metro Type', 'school_metro_type')\
.withColumnRenamed('School Percentage Free Lunch', 'school_percent_free_lunch').withColumnRenamed('School State', 'school_state')\
.withColumnRenamed('School Zip', 'school_zip').withColumnRenamed('School City', 'school_city').withColumnRenamed('School County', 'school_county')\
.withColumnRenamed('School District', 'school_district').withColumnRenamed('Teacher Prefix', 'teacher_prefix')\
.withColumnRenamed('Teacher First Project Posted Date', 'teacher_first_proj').withColumnRenamed('Resource Item Name', 'resource_item_name')\
.withColumnRenamed('Resource Quantity', 'resource_quantity').withColumnRenamed('Resource Unit Price', 'resource_unit_price')\
.withColumnRenamed('Resource Vendor Name', 'resource_vendor_name')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#cast timestamps to all time columns
df_renamed = df_renamed.withColumn("donation_received_date",to_date(unix_timestamp(col("donation_received_date"), "yyyy-MM-dd").cast("timestamp")))
df_renamed = df_renamed.withColumn("project_posted_date",to_date(unix_timestamp(col("project_posted_date"), "yyyy-MM-dd").cast("timestamp")))
df_renamed = df_renamed.withColumn("project_exp_date",to_date(unix_timestamp(col("project_exp_date"), "yyyy-MM-dd").cast("timestamp")))
df_renamed = df_renamed.withColumn("project_fully_funded_date",to_date(unix_timestamp(col("project_fully_funded_date"), "yyyy-MM-dd").cast("timestamp")))
df_renamed = df_renamed.withColumn("teacher_first_proj",to_date(unix_timestamp(col("teacher_first_proj"), "yyyy-MM-dd").cast("timestamp")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#create new datetime columns for future data exploration
df_renamed = df_renamed.withColumn('teacher_date_exp_dif',datediff(df_renamed.project_exp_date.cast('date'),df_renamed.teacher_first_proj.cast('date')))
df_renamed= df_renamed.withColumn('project_days',datediff(df_renamed.project_exp_date.cast('date'),df_renamed.project_posted_date.cast('date')))

df_renamed = df_renamed.withColumn('post_date_day', date_format(col('project_posted_date'), "E"))
df_renamed = df_renamed.withColumn('post_date_month', month(df_renamed['project_posted_date']))
df_renamed = df_renamed.withColumn('post_date_year', year(df_renamed['project_posted_date']))

df_renamed = df_renamed.withColumn('donation_day', date_format(col('donation_received_date'), "E"))
df_renamed = df_renamed.withColumn('donation_month', month(df_renamed['donation_received_date']))
df_renamed = df_renamed.withColumn('donation_year', year(df_renamed['donation_received_date']))

df_renamed = df_renamed.withColumn('t_proj_post_day', date_format(col('teacher_first_proj'), "E"))
df_renamed = df_renamed.withColumn('t_proj_post_month', month(df_renamed['teacher_first_proj']))
df_renamed = df_renamed.withColumn('t_proj_post_year', year(df_renamed['teacher_first_proj']))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
#cast column types for ints and floats that weren't inferred at import

df_renamed = df_renamed.withColumn('school_percent_free_lunch',df_renamed.school_percent_free_lunch.cast('float'))
df_remamed = df_renamed.withColumn('donor_cart_sequence',df_renamed.donor_cart_sequence.cast('int'))
df_renamed = df_renamed.withColumn('donation_amount',df_renamed.donation_amount.cast('float'))
df_renamed = df_renamed.withColumn('teacher_project_posted_seq',df_renamed.teacher_project_posted_seq.cast('int'))
df_renamed = df_renamed.withColumn('project_cost',df_renamed.project_cost.cast('float'))
df_renamed = df_renamed.withColumn('resource_quantity',df_renamed.resource_quantity.cast('float'))
df_renamed = df_renamed.withColumn('resource_unit_price',df_renamed.resource_unit_price.cast('float'))
df_renamed = df_renamed.withColumn('resource_unit_price',df_renamed.resource_unit_price.cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Modeling - Recommendation Engines

In [11]:
#import libraries for modeling
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Pre-Processing Data

In [12]:
#create new table that will bring in donor_id, project_id, and donation amount per donor per project
df3 = df_renamed.select(df_renamed['donor_id'], df_renamed['project_id'], df_renamed['donation_amount'])
#sum the donation amount
donor_project_table = df3.groupBy("donor_id", "project_id").sum('donation_amount')
#rename column to reflect sum of donation amount
donor_project_table = donor_project_table.withColumnRenamed("sum(donation_amount)", 'donation_sum')
#find log values for all donation sums
donor_project_table = donor_project_table.withColumn("donation_log", log("donation_sum"))

#create new user table that will take distinct donor ID's and map them to an integer ID
users = donor_project_table.select("donor_id").distinct()
users = users.coalesce(1)
users = users.withColumn("user_int", monotonically_increasing_id()).persist()

#create new projects table that will take distinct project ID's and map them to an integer ID
projects = donor_project_table.select("project_id").distinct()
projects = projects.coalesce(1)
projects = projects.withColumn("project_int", monotonically_increasing_id()).persist()

#join the users and projects table back to the original table with donation sum and donation log
prepped_df = donor_project_table.join(users, "donor_id", "left").join(projects, "project_id", "left")

#create final table for modeling with only the new int donor ID, new int project ID and the donation log
cfm_df = prepped_df.select(prepped_df["user_int"], prepped_df["project_int"], prepped_df["donation_log"])
cfm_df= cfm_df.withColumnRenamed('user_int', 'donor_int_ID').withColumnRenamed('project_int', 'project_int_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#cast all columns to int to ensure they're all the correct format for ALS modeling
cfm_df= cfm_df.withColumn('donor_int_ID', cfm_df.donor_int_ID.cast('int'))
cfm_df= cfm_df.withColumn('project_int_id', cfm_df.project_int_id.cast('int'))
cfm_df= cfm_df.withColumn('donation_log', cfm_df.donation_log.cast('int'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#double check data types before modeling
cfm_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- donor_int_ID: integer (nullable = true)
 |-- project_int_id: integer (nullable = true)
 |-- donation_log: integer (nullable = true)

In [16]:
cfm_df.agg(min(col("donation_log")), max(col("donation_log"))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+
|min(donation_log)|max(donation_log)|
+-----------------+-----------------+
|               -4|               15|
+-----------------+-----------------+

##### Alternating Least Squares (ALS) Model

In [15]:
#random split
training, testing = cfm_df.randomSplit([0.6, 0.4])

#create base ALS model
als = ALS(userCol="donor_int_ID", itemCol="project_int_id", ratingCol="donation_log", coldStartStrategy="drop", nonnegative = True, implicitPrefs = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#fit model to training data
model = als.fit(training)

#fit model to testing data
predictions = model.transform(testing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#create evaluator
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = 'donation_log', predictionCol = "prediction")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#run evaluator on base model predictions - testing dataset, find initial RMSE
RMSE = evaluator.evaluate(predictions)
print(RMSE)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.9441559327012328

In [19]:
#create param builder
paramGrid = ParamGridBuilder() \
    .addGrid(als.rank, [15])\
    .build()

#build cross validator
cv = CrossValidator(estimator=als, estimatorParamMaps = paramGrid,
                   evaluator = evaluator,
                   numFolds = 5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#fit training data to cross validator
model = cv.fit(training)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#determine best parameters based off of cross validator
best_model = model.bestModel
predictions = best_model.transform(testing)
rmse = evaluator.evaluate(predictions)

print("Best Model")
print("RMSE: ", rmse)
print("Rank: ", best_model.rank)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best Model
RMSE:  1.9401127004230694
Rank:  15

In [30]:
#create base ALS model
als = ALS(userCol="donor_int_ID", itemCol="project_int_id", ratingCol="donation_log", coldStartStrategy="drop", nonnegative = True, implicitPrefs = False, rank = 15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
#fit model to training data
model = als.fit(training)

#fit model to testing data
predictions = model.transform(testing)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
cfm_df.agg(min(col("donation_log")), max(col("donation_log")).show()

VBox()

An error was encountered:
Session 1 did not reach idle status in time. Current status is busy.
